In [1]:
import autograd as autograd 
from autograd import numpy as np 
import matplotlib.pyplot as plt 


# Projet Maths info
## Courbes de niveau 



# Déceler une amorce
**Méthode de Newton à 1D pour des fonctions de 1 variable**
- Soit $f: [0,1]² \to \mathbb{R}$ , considérons $ g:[0,1] \to \mathbb{R} , y \mapsto f(x,y)$ la fonction de une variable.
- Résolvons l'équation $g(y)=c$ sur $[0,1]$ :
    - Pour ce faire utilisons la méthode de Newton qui nous dit, en supposant que la fonction est différentiable et que sa dérivéee ne s'annule pas sur $[0,1]$,  que la suite $(_k)_\mathbb{N}$ définie par $ y_{k+1}=y_{k} -\frac{g(x_k)}{g'(x_{k})}$ est convrgente et de limite y solution de l'équation $g(y)=c$
- On implémente ainsi l'algorithme de résolution d'équation en construisant itérativement la suite.
- On considère la limite atteinte lorsque la distance entre deux $x_k$ successifs est plus petit qu'un certain $\epsilon$ fixé


In [2]:
def find_seed(g,c=0,eps=2**(-26)):
    if (g([0,1])-c)*(g([0,0])-c)<0:
        return None
    else : 
        gradg=autograd.grad(g)
        x_0=5
        x=x_0-(g(np.array([0.,x_0]))-c)/gradg(np.array([0.,x_0]))[1]
        while abs(x-x_0)>eps:
            x_0=x
            x=x_0-(g(np.array([0,x_0]))-c)/gradg(np.array([0,x_0]))[1]
        return [0,x] 

In [3]:
g= lambda X : (X[1])**2 +(X[0])**2

In [4]:
find_seed(g,1)

[0, 1.0]